# Tensorflow basics

In deze notebook worden een aantal basisconcepten van tensorflow toegelicht en bekeken aan de hand van een aantal voorbeelden.

## Tensors

Het basisconcept van tensorflow is een tensor.
Dit is een multi-dimensionele rij met een uniform datatype of dtype.
Dit concept komt heel sterk overeen met een array van Numpy.
Alle tensors zijn onwijzigbaar of immutable, net zoals spark dataframes.
Dit houdt in dat hun waarden niet kunnen gewijzigd worden.
Alle operaties op een tensor maken dus een nieuwe tensor aan.

Een tensor aanmaken kan door middel van de constant functie.

In [1]:
import tensorflow as tf
import timeit
from datetime import datetime
import numpy as np

In [2]:
tensor1 = tf.constant(4)
print(tensor1)

tensor2 = tf.constant([2.0, 3.0, 4.0])
print(tensor2)

tensor3 = tf.constant([[1, 2, 3], [4, 5, 6], [7, 8, 9]], dtype=tf.float32)
print(tensor3)

tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor([2. 3. 4.], shape=(3,), dtype=float32)
tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]], shape=(3, 3), dtype=float32)


Op tensors kunnen de volgende operaties uitgevoerd worden
* add
* multiply
* matrix multiplication
* en andere wiskunde operaties zoals softmax, maximum vinden, ... (zie hiervoor ook de [math package](https://www.tensorflow.org/api_docs/python/tf/math))

In [3]:
#add
print(tf.add(tensor1, 3))
print(tf.add(tensor2, 3))

#multiply
print(tf.multiply(3, tensor3))
print(3 * tensor3)

#matrix multiplication
print(tf.matmul(tensor3, tensor3))
print(tensor3 @ tensor3)

#find maximum
print(tf.reduce_max(tensor3))

#find index of maximum
print(tf.math.argmax(tensor3))

#compute softmax for each element in tensor
print(tf.nn.softmax(tensor3))

tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor([5. 6. 7.], shape=(3,), dtype=float32)
tf.Tensor(
[[ 3.  6.  9.]
 [12. 15. 18.]
 [21. 24. 27.]], shape=(3, 3), dtype=float32)
tf.Tensor(
[[ 3.  6.  9.]
 [12. 15. 18.]
 [21. 24. 27.]], shape=(3, 3), dtype=float32)
tf.Tensor(
[[ 30.  36.  42.]
 [ 66.  81.  96.]
 [102. 126. 150.]], shape=(3, 3), dtype=float32)
tf.Tensor(
[[ 30.  36.  42.]
 [ 66.  81.  96.]
 [102. 126. 150.]], shape=(3, 3), dtype=float32)
tf.Tensor(9.0, shape=(), dtype=float32)
tf.Tensor([2 2 2], shape=(3,), dtype=int64)
tf.Tensor(
[[0.09003057 0.24472848 0.6652409 ]
 [0.09003057 0.24472848 0.6652409 ]
 [0.09003057 0.24472848 0.6652409 ]], shape=(3, 3), dtype=float32)


**Shapes**

De dimensies van een tensor zijn zeer belangrijk.
Bij communicatie over de tensors worden de volgende termen gebruikt:
* Axis: Dit is een specifieke dimensie van de tensor
* Rank: Het aantal dimensies waarover de tensor beschikt
    * Eenvoudig getal: 0
    * Een enkelvoudige rij: 1
    * Een matrix: 2
* Shape: De lengte van elk van de dimensies van de tensor. **Let op**: Buiten een aantal uitzondering moet een tensor steeds rechthoekig zijn. Dit wil zeggen dat elk element op een axis dezelfde lengte moet hebben.
* Size: Het totaal aantal elementen in de tensor. Dit is het product van de lengtes van elke dimensie

Deze waarden van een tensor kunnen als volgt berekend worden:

In [4]:
print("Type of every element:", tensor3.dtype)
print("Number of axes:", tensor3.ndim)
print("Shape of tensor:", tensor3.shape)
print("Elements along axis 0 of tensor:", tensor3.shape[0])
print("Elements along the last axis of tensor:", tensor3.shape[-1])
print("Total number of elements (3*3): ", tf.size(tensor3))

Type of every element: <dtype: 'float32'>
Number of axes: 2
Shape of tensor: (3, 3)
Elements along axis 0 of tensor: 3
Elements along the last axis of tensor: 3
Total number of elements (3*3):  tf.Tensor(9, shape=(), dtype=int32)


**Indexing:**

Tensorflow maakt gebruik van de standaard python indexing regels:
* Indexes beginnen op 0
* Negatieve indexes tellen vanaf het einde
* Dubbelpunt wordt gebruikt om slices of bereiken te kiezen volgens start:stop:end

In [5]:
t = tf.constant([0, 1, 1, 2, 3, 5, 8, 13, 21, 34])
print(t.numpy())
# 1 axis geselecteerd dus rank is 1 kleiner
print("First:", t[0].numpy())
print("Second:", t[1].numpy())
print("Last:", t[-1].numpy())
# gebruik maken van de slices zorgt ervoor dat rank gelijk blijft
print("Everything:", t[:].numpy())
print("Before 4:", t[:4].numpy())
print("From 4 to the end:", t[4:].numpy())
print("From 2, before 7:", t[2:7].numpy())
print("Every other item:", t[::2].numpy())
print("Reversed:", t[::-1].numpy())

# multi-axis indexing
print()
print(tensor3.numpy())
print(tensor3[1, 1].numpy())
print("Second row:", tensor3[1, :].numpy())
print("Second column:", tensor3[:, 1].numpy())
print("Last row:", tensor3[-1, :].numpy())
print("First item in last column:", tensor3[0, -1].numpy())
print("Skip the first row:")
print(tensor3[1:, :].numpy(), "\n")

[ 0  1  1  2  3  5  8 13 21 34]
First: 0
Second: 1
Last: 34
Everything: [ 0  1  1  2  3  5  8 13 21 34]
Before 4: [0 1 1 2]
From 4 to the end: [ 3  5  8 13 21 34]
From 2, before 7: [1 2 3 5 8]
Every other item: [ 0  1  3  8 21]
Reversed: [34 21 13  8  5  3  2  1  1  0]

[[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]
5.0
Second row: [4. 5. 6.]
Second column: [2. 5. 8.]
Last row: [7. 8. 9.]
First item in last column: 3.0
Skip the first row:
[[4. 5. 6.]
 [7. 8. 9.]] 



**Manipuleren van een tensor**

Reshaping of het aanpassen van de shape van een tensor kan vaak heel handig zijn.
Hiervoor kan je de reshape functie gebruiken. 
Deze operatie is heel rekenefficient omdat de data ongewijzigd blijft.

Voor ML moet er vaak een matrix van features (figuren) omgezet worden naar een 1-dimensionale vector/rij/tensor om het te kunnen aanbieden aan een neuraal netwerk. Deze operatie wordt ook flatten genoemd.

Typisch is het enkel aangeraden om een reshape te doen om dimensies met een lengte van 1 weg te laten of naburige axis te combineren of te splitsen. Indien je meer willekeurige zaken doet gaan de resultaten niet altijd bruikbaar zijn omdat de volgorde van dimensies niet gerespecteerd blijft.

In [6]:
x = tf.constant([[1], [2], [3]])
print(x.numpy())
print(x.shape)
print()

x = tf.reshape(x, [1,3])
print(x.numpy())
print(x.shape)

#flatten
# A `-1` passed in the `shape` argument says "Whatever fits".
print(tf.reshape(tensor3, [-1]).numpy())

[[1]
 [2]
 [3]]
(3, 1)

[[1 2 3]]
(1, 3)
[1. 2. 3. 4. 5. 6. 7. 8. 9.]


Daarnaast zijn er nog een aantal interessante kenmerken van tensors die zelf verder bestudeerd kunnen worden.
Enkele voorbeelden hiervan zijn:
* Broadcasting
* Data types
* Sparse tensors
* Ragged tensors
* String tensors

**Oefening:**

Maak twee tensors aan met respectievelijk een shape van 3x4 en 4x3.
* Bereken het product van beide matrices. Kan dit product in beide richtingen berekend worden?
* Wat is het gemiddelde van de eerste tensor?
* Wat is het minimum van de tweede tensor?
* Bereken het negatieve van de eerste tensor. Dit betekend dat elk element vermenigvuldigd wordt met -1.

In [7]:
x = tf.constant(range(12))
tensor1 = tf.reshape(x, [3,4])
tensor2 = tf.reshape(x, [4,3])
print(tensor1.numpy())
print(tensor2.numpy())

print(tensor1 @ tensor2)
print(tensor2 @ tensor1)

print(tf.math.reduce_mean(tensor1))
print(tf.math.reduce_min(tensor2))
print(tf.math.negative(tensor1))
print(tensor1 * -1)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]
[[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]]
tf.Tensor(
[[ 42  48  54]
 [114 136 158]
 [186 224 262]], shape=(3, 3), dtype=int32)
tf.Tensor(
[[ 20  23  26  29]
 [ 56  68  80  92]
 [ 92 113 134 155]
 [128 158 188 218]], shape=(4, 4), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(
[[  0  -1  -2  -3]
 [ -4  -5  -6  -7]
 [ -8  -9 -10 -11]], shape=(3, 4), dtype=int32)
tf.Tensor(
[[  0  -1  -2  -3]
 [ -4  -5  -6  -7]
 [ -8  -9 -10 -11]], shape=(3, 4), dtype=int32)


## Variables

Met tensors kan je reeds heel wat berekeningen uitvoeren.
Echter is het onmogleijk om een persistente state te hebben die gemanipuleerd wordt door het programma omdat ze immutable zijn.
Deze persistente state kan bijvoorbeeld een tensor van de gewichten zijn in het model of andere parameters die gewijzigd of getrained moeten worden.

Om deze state bij te houden moet er gebruikt gemaakt worden van een Variable.
Deze kan aangemaakt worden door gebruik te maken van de klasse tf.Variable().
Een object van deze klasse stelt een tensor voor dat wel kan aangepast worden door er operaties op uit te voeren.

In [8]:
my_tensor = tf.constant([[1.0, 2.0], [3.0, 4.0]])
my_variable = tf.Variable(my_tensor)

# Variables can be all kinds of types, just like tensors
bool_variable = tf.Variable([False, False, False, True])
complex_variable = tf.Variable([5 + 4j, 6 + 1j])

print("Shape: ", my_variable.shape)
print("DType: ", my_variable.dtype)
print("As NumPy: ", my_variable.numpy())

Shape:  (2, 2)
DType:  <dtype: 'float32'>
As NumPy:  [[1. 2.]
 [3. 4.]]


Zoals hierboven aangehaald moeten er specifieke operaties gebruikt worden om de waarde van de tensor aan te passen.
Een aantal mogelijke operaties hiervoor zijn:
* tf.Variable.assign()
* tf.Variable.assign_add()
* tf.Variable.assign_sub()
* Andere functies kan je [hier](https://www.tensorflow.org/api_docs/python/tf/Variable) vinden.

Let op dat hierbij de aangemaakt tensor herbruikt wordt. Hierdoor is het dus niet mogelijk om een andere size in te stellen voor de variable.

In de code hieronder zie je een aantal code voorbeelden voor te werken met variabelen:

In [9]:
a = tf.Variable([2.0, 3.0])
# This will keep the same dtype, float32
a.assign([1, 2]) 
# Not allowed as it resizes the variable: 
try:
    a.assign([1.0, 2.0, 3.0])
except Exception as e:
    print(f"{type(e).__name__}: {e}")
    
a = tf.Variable([2.0, 3.0])
# Create b based on the value of a
b = tf.Variable(a)
a.assign([5, 6])

# a and b are different
print(a.numpy())
print(b.numpy())

# There are other versions of assign
print(a.assign_add([2,3]).numpy())  # [7. 9.]
print(a.assign_sub([7,9]).numpy())  # [0. 0.]

ValueError: Cannot assign value to variable ' Variable:0': Shape mismatch.The variable shape (2,), and the assigned value shape (3,) are incompatible.
[5. 6.]
[2. 3.]
[7. 9.]
[0. 0.]


Voor performantie probeert Tensorflow steeds de tensors en variabelen op het snelste toestel te plaatsen dat compatibel is met zijn datatype. 
Dit betekend dat de meeste variabelen op de **GPU** bewaard worden indien er 1 beschikbaar is.
Dit gedrag kan aangepast worden doro middel van de tf.device functie.
Indien je hiervan gebruik maakt kan je er ook voor zorgen dat de locatie ingesteld wordt op 1 toestel en de uitvoering op een ander gedaan wordt.
Dit introduceert wel een delay omdat de data gekopieerd moet worden naar het andere toestel.
Dit wordt soms gedaan als je meerdere GPU's gebruikt maar slechts 1 kopie wilt van de variabelen.

In [10]:
with tf.device('CPU:0'):
    a = tf.Variable([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
    b = tf.Variable([[1.0, 2.0, 3.0]])

with tf.device('GPU:0'):
    # Element-wise multiply
    k = a * b

print(k)

tf.Tensor(
[[ 1.  4.  9.]
 [ 4. 10. 18.]], shape=(2, 3), dtype=float32)


## Eager vs graph execution

In de code hierboven is steeds gebruik gemaakt van zogenoemde **eager execution**. 
Dit houdt in dat de code uitgevoerd wordt op dezelfde manier dat de python code uitgevoerd wordt, namelijk:
* Lijn per lijn wordt de code geinterpreerd
* Operatie na operatie wordt uitgevoerd
* De resultaten worden na elke operatie teruggegeven aan de python interpreter

Ook al heeft deze eager execution een aantal voorbeelden (met name dat het vlotter programmeert in python) zijn er ook een aantal problemen.
Het grootste probleem is dat je code enkel kan uitgevoerd worden op een systeem dat python kan draaien.
Daarnaast is het ook niet het meest performante systeem omdat de interpreter van python geen code kan optimaliseren doordat het lijn per lijn werkt.
Een alternatieve manier van uitvoering dat beschikbaar gesteld wordt door tensorflow wordt **graph execution** genoemd.
Dit houdt in dat je in python code een graaf opsteld van de operaties die voor een bepaalde input moeten uitgevoerd worden.
Tensorflow kan dan alle operaties in de volledige graaf in 1 keer en in parallel uitvoeren.
Meer informatie over deze manier van code uitvoeren kan je [hier](https://www.tensorflow.org/guide/intro_to_graphs) vinden.

In [11]:
# Define a Python function.
def a_regular_function(x, y, b):
    x = tf.matmul(x, y)
    x = x + b
    return x

# maak van de python functie een graph functie by wrapping it
a_function_that_uses_a_graph = tf.function(a_regular_function)

# test de twee manieren
x1 = tf.constant([[1.0, 2.0]])
y1 = tf.constant([[2.0], [3.0]])
b1 = tf.constant(4.0)

orig_value = a_regular_function(x1, y1, b1).numpy()
tf_function_value = a_function_that_uses_a_graph(x1, y1, b1).numpy()
assert(orig_value == tf_function_value)

# tweede manier om een graph functie aan te maken
# door middel van decorator (annotatie) @
@tf.function
def outer_function(x):
    y = tf.constant([[2.0], [3.0]])
    b = tf.constant(4.0)

    return a_regular_function(x, y, b)

outer_function(tf.constant([[1.0, 2.0]])).numpy()

array([[12.]], dtype=float32)

Een eerste belangrijke opmerking is dat een graph gemaakt wordt voor elke unieke set van inputs (dus op basis de specifieke waarden).
Dit wordt ook wel de handtekening of (input) signature genoemd van de graph.
Dit is belangrijk omdat op deze manier de resultaten gecached kunnen worden en de optimalisatie niet steeds opnieuw moet gebeuren.

De omzetting van eager execution naar graph execution gaat vrij automatisch en probleemloos.
De meeste standaard python code wordt zonder problemen en volledig omgezet.
Een eerste belangrijke opmerking is hoe de graph execution werkt.
Bekijk hiervoor de output van onderstaande voorbeeld.

In [12]:
@tf.function
def get_MSE(y_true, y_pred):
    print("Calculating Mean Square Error!")
    sq_diff = tf.pow(y_true - y_pred, 2)
    return tf.reduce_mean(sq_diff)

y_true = tf.random.uniform([5], maxval=10, dtype=tf.int32)
y_pred = tf.random.uniform([5], maxval=10, dtype=tf.int32)
print(y_true)
print(y_pred)

# force eager execution
#tf.config.run_functions_eagerly(True)

error = get_MSE(y_true, y_pred)
error = get_MSE(y_true, y_pred)
error = get_MSE(y_true, y_pred)

tf.config.run_functions_eagerly(False)


tf.Tensor([6 1 4 2 0], shape=(5,), dtype=int32)
tf.Tensor([2 2 2 5 3], shape=(5,), dtype=int32)
Calculating Mean Square Error!


Het valt op dat als je bovenstaande code uitvoert als graph execution, dat de output slechts 1 keer uitgeprint wordt. 
Dit komt omdat de graph geoptimaliseerd wordt door het "tracing" proces.
De print wordt hierdoor niet meegenomen omdat het geen deel uitmaakt van de tensorflow operaties en ook geen impact heeft.
De print functie wordt ook wel een python side effect genoemd.
Als je toch steeds een print wil uitvoeren moet je gebruik maken van de functie **tf.print()**.

Wanneer de graph execution uitgeschakeld wordt door middel van de run_functions_eagerly functie  dan zien we dat de print wel drie keer uitgevoerd wordt.
Typisch worden bij graph executions enkel de volgende functies uitgevoerd:
* De return value
* Functies met well-known side effects zoals
    * tf.print
    * tf.debugging (asserts bvb)
    * aanpassingen van tf.Variable objecten
    
Dit type gedrag wordt ook non-strict execution genoemd omdat niet alle stappen van het programma doorlopen worden.
Dit heeft ook als gevolg dat het kan zijn dat exceptions niet opgeworpen worden als het gebeurt in een functie die niet opgeroepen wordt.
**Reken dus niet op het opwerpen van een exception**.
Een voorbeeld hiervan kan bestudeerd worden in onderstaande voorbeeld.

In [13]:
# eager execution
def unused_return_eager(x):
    # Get index 1 zou foutmelding moeten opwerpen als de lengte van x 1 is
    tf.gather(x, [1]) # unused 
    return x

try:
    print(unused_return_eager(tf.constant([0.0])))
except tf.errors.InvalidArgumentError as e:
    # All operations are run during eager execution so an error is raised.
    print(f'{type(e).__name__}: {e}')

tf.Tensor([0.], shape=(1,), dtype=float32)


In [14]:
# graph execution
@tf.function
def unused_return_graph(x):
    tf.gather(x, [1]) # unused
    return x

# Only needed operations are run during graph execution. The error is not raised.
print(unused_return_graph(tf.constant([0.0])))

tf.Tensor([0.], shape=(1,), dtype=float32)


**Performantie boost**

Gebruik maken van graph execution heeft typisch een performantie winst tot gevolg, afhankelijk van het type van berekening dat je uitvoert.
Hieronder staat een voorbeeld waarbij een aantal matrixberekeningen uitgevoerd worden en de uitvoeringstijd voor zowel eager als graph execution wordt berekend.

In [15]:
# eager execution

x = tf.random.uniform(shape=[10, 10], minval=-1, maxval=2, dtype=tf.dtypes.int32)

def power(x, y):
    result = tf.eye(10, dtype=tf.dtypes.int32)
    for _ in range(y):
        result = tf.matmul(x, result)
    return result

print("Eager execution:", timeit.timeit(lambda: power(x, 1000), number=1000))

Eager execution: 27.72609779999766


In [16]:
# graph execution
power_as_graph = tf.function(power)
print("Graph execution:", timeit.timeit(lambda: power_as_graph(x, 1000), number=1000))

Graph execution: 2.3189291000016965


Let wel op dat deze performantie boost enkel ervaren wordt wanneer een bepaalde functie meermaals uitgevoerd wordt.
De eerste keer dat de functie uitgevoerd wordt is er extra rekentijd nodig om de graph op te bouwen en te optimaliseren (**tracing** genoemd).
Deze extra kost op vlak van rekentijd wordt echter snel terugverdiend door de volgende functies.

**Best practices voor graph execution**

* **Wissel tussen eager en graph execution** om vast te stellen op welk punt de modes wijzigen.
* Maak tf.Variables aan buiten de python functie en wijzig ze binnen de functie. Dit geldt ook voor andere objecten zoals layers, Models, optimizers, ...
* Vermijd functies te schrijven die afhangen van standaard python variabelen behalve tf.Variables en Keras objecten.
* Geef de voorkeur aan tensors en en tensorflow types als inputs van functies
* Plaats zoveel mogelijk bewerkingen in tf.functions om de performantiewinst te optimaliseren.